In [1]:
import utils
import os, re
import jieba
import numpy as np
from settings import Settings

In [2]:
setting = Settings()

In [3]:
clean_dir = os.path.join('data', 'clean')
helper_dir = os.path.join('data', 'helper')
dataset_dir = os.path.join('data', 'dataset')

In [28]:
y_pred_start = utils.pickleLoader('y_pred_start.pkl')
y_pred_end = utils.pickleLoader('y_pred_end.pkl')
print(np.array(y_pred_start).shape)
print(np.array(y_pred_end).shape)

loading file y_pred_start.pkl
finished loading y_pred_start.pkl
loading file y_pred_end.pkl
finished loading y_pred_end.pkl
(3205, 50)
(3205, 50)


In [29]:
validation_set_raw = utils.pickleLoader(os.path.join(dataset_dir, 'validation_raw_plen50.pkl'))

loading file data\dataset\validation_raw_plen50.pkl
finished loading data\dataset\validation_raw_plen50.pkl


In [23]:
idx = 15
print(validation_set_raw[idx][0][0])
print(validation_set_raw[idx][0][1])
print(validation_set_raw[idx][-1])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
{'answer_idx': [], 'query_id': 11613, 'answer': '圆通快递', 'query': '衡水学院快递', 'passage': {'passage_text': '院几号楼,几室,快递员来了会给你打电话,你去学院门口等就可以了,你说的收发室指的是平信快递没这么麻烦', 'url': 'http://zhidao.baidu.com/question/188098287'}}


In [32]:
def get_answer_list(db_test, y_pred_start, y_pred_end, caseStudy=False):
    def GetAnswer(passage, start, end):
        return passage[start:end + 1]
    
    rankScore = {}
    
    for idx in range(len(db_test)):
        test_details = db_test[idx][-1]
        query_id = str(test_details['query']['query_id'])
        query = test_details['query']['query_text']
        answer = test_details['answer']
        passage = test_details['passage']['passage_text']
        thisMax = -1
        canswer = "NaN**"
        for i in range(setting.passage_len):
            for j in range(i, setting.passage_len):
                if j - i > 10:
                    break
                score = y_pred_start[idx][i] * y_pred_end[idx][j]
                if score > thisMax or canswer == "NaN**":
                    temp = GetAnswer(passage, i, j)
                    if temp in query: continue
                    thisMax, canswer = score, temp
        rankScore[query_id] = rankScore.get(query_id, {'query': query,
                                                       'answer': answer,
                                                       'answer_rank': []})
        rankScore[query_id]['answer_rank'].append({
                                                    'score': score,
                                                    'answer': canswer,
                                                    'passage': passage
                                                   })
        
    for i, j in rankScore.items():
        rankScore[i]['answer_rank'] = sorted(j['answer_rank'], key=lambda x: x['score'], reverse=True)
        rankScore[i]['pred_ans'] = [item['answer'] for item in rankScore[i]['answer_rank']]
    return rankScore

In [33]:
rank_score = get_answer_list(validation_set_raw, y_pred_start, y_pred_end)

In [34]:
def evaluate(answer_list):
    tr = 0
    tot = len(answer_list)
    for _, item in answer_list.items():
        t_answer = item['answer']
        ans = item['pred_ans']
        if t_answer in ans:
            tr += 1
            print(t_answer)
            print(ans)
            print('='*20)
    print('tr: ', tr)
    print('tr / tot: ', tr / tot)

In [35]:
evaluate(rank_score)

聂耳
['中华人民共和国国', '克族的但是可以听', '克族的但是可以听', '克族的但是可以听', '克族的但是可以听', '中华', '中华', '阅读(:0:):中', '/zhwiki', '/zhwiki', '首也是中国历', '作曲:', '聂耳']
美国
['计算机诞生于哪个国', '计算机诞生于哪个国', '方要', '方要', '8000多个电子', '8000多个电子', '000多倍,', '000多倍,', '000多倍,', '000多倍,', '是哪', '是哪', '1946', '美国', 'us', 'us']
丁
['谜底.', '时', '时', '怎样', '我猜谜底是', '怎样', '查找', '"打"', '丁', '丁', '宁']
山西省
['吕梁虽然也很差', '岚相望;南部与', '岚相望;南部与', '忻州属于山西', '忻州属于山西', '吧:只', '吧:只', '是哪里', '是哪里', '是哪里', '是哪里', '卧龙沟', '山西省', '山西']
鲁迅
['一篇报', '一篇报', '才,浙江绍兴', '才,浙江绍兴', '吃人者的本质:"狮', '吃人者的本质:"狮', '《狂人日记》是鲁', '《', '的描写', '会.陈老五', '了一会', '了一会', '鲁迅']
松下
['手机', '上次', '上次', '请问', '-02-10.视', '-02-10.视', 'panasonic灯', '牌子是panas', '是:松', '牌子是', '松下']
冲
['矗驰匪', '矗驰匪', '是"', '是"', '字加一笔是', '把"', '逆推', '逆推', '4', '4', ',论坛', '冲']
水星
['八大行星中最靠近太', '娇小的行星—', '水星', '太阳)水)', '太阳)水)', '太阳)水)', '太阳)水)', '%,重量比地球', '%,重量比地球', '今年', '原来这两']
湖北省
['有关规定,上', '培训相', ':当阳市位于湖', '你好,我是当阳人', '[摘要]:正位', '带.地势西北高,', '带.地势西北高,', '们的', '们的', '玉泉寺位于湖', '生"', '湖北省']
春季
['好想', '原为', '和暑期

In [37]:
keys = list(rank_score.keys())
print(len(keys))

251


In [38]:
print(rank_score[keys[78]])

{'answer_rank': [{'score': 0.001650687, 'answer': '手游<'}, {'score': 0.0016470924, 'answer': '手游巧思猜灯谜答'}, {'score': 0.0016205538, 'answer': '手游巧思猜灯谜活'}, {'score': 0.0016172873, 'answer': '手游元'}, {'score': 0.0016159853, 'answer': '3>,'}, {'score': 0.0015866966, 'answer': '手游巧思猜灯谜所'}, {'score': 0.0015531802, 'answer': '玩游戏'}, {'score': 0.0015147683, 'answer': '源:齐'}, {'score': 0.0015084198, 'answer': '手游巧思猜灯谜活动'}, {'score': 0.0014134306, 'answer': '西为玩家'}], 'answer': '元宵节', 'pred_ans': ['手游<', '手游巧思猜灯谜答', '手游巧思猜灯谜活', '手游元', '3>,', '手游巧思猜灯谜所', '玩游戏', '源:齐', '手游巧思猜灯谜活动', '西为玩家'], 'query': '梦幻西游手游巧思猜灯谜'}


In [36]:
a = {}

In [37]:
print(a.get('h'))

None
